In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

# Load the dataset
train_dataset = datasets.ImageFolder(root='D:\sdp\ll-first\dataset\training', transform=transform)
test_dataset = datasets.ImageFolder(root='D:\sdp\ll-first\dataset\testing', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the custom model
class CustomVGG16(nn.Module):
    def __init__(self, num_classes=3):
        super(CustomVGG16, self).__init__()
        vgg16 = models.vgg16(pretrained=True)
        self.features = vgg16.features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Instantiate the model
model = CustomVGG16(num_classes=3).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Training loop
def train(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Testing loop
def test(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Train the model
train(model, train_loader, criterion, optimizer, epochs=10)

# Test the model
test(model, test_loader)

<>:20: SyntaxWarning: invalid escape sequence '\s'
<>:21: SyntaxWarning: invalid escape sequence '\s'
<>:20: SyntaxWarning: invalid escape sequence '\s'
<>:21: SyntaxWarning: invalid escape sequence '\s'
C:\Users\anmol\AppData\Local\Temp\ipykernel_21456\3279755217.py:20: SyntaxWarning: invalid escape sequence '\s'
  train_dataset = datasets.ImageFolder(root='D:\sdp\ll-first\dataset\training', transform=transform)
C:\Users\anmol\AppData\Local\Temp\ipykernel_21456\3279755217.py:21: SyntaxWarning: invalid escape sequence '\s'
  test_dataset = datasets.ImageFolder(root='D:\sdp\ll-first\dataset\testing', transform=transform)


Using device: cuda


C:\Users\anmol\AppData\Local\Temp\ipykernel_21456\3279755217.py:20: SyntaxWarning: invalid escape sequence '\s'
  train_dataset = datasets.ImageFolder(root='D:\sdp\ll-first\dataset\training', transform=transform)
C:\Users\anmol\AppData\Local\Temp\ipykernel_21456\3279755217.py:21: SyntaxWarning: invalid escape sequence '\s'
  test_dataset = datasets.ImageFolder(root='D:\sdp\ll-first\dataset\testing', transform=transform)


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'D:\\sdp\\ll-first\\dataset\training'